# Creating Distance Matrices

In [6]:
## Modules
import geopandas as gpd
import pandas as pd
import numpy as np

import json
import urllib.request
import geopandas as gpd
import pandas as pd
import numpy as np
#import geopy.distance

In [64]:

#cars_by_county = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header=0)
#cars_by_county

## All Locations (FQHCs and PPs)

### Defining Functions

In [ ]:
health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0)
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)

In [ ]:
## Utils
'''health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0)
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)'''

def get_county(site, health_centers):
    return health_centers.loc[site]['county']
def get_pop(county_name):
    # Remove "County" from the end and convert to lowercase for matching
    county_normalized = county_name.lower().replace(" county", "").strip()
    
    # Find matching county in county_pop dataframe (case-insensitive)
    county_match = county_pop[county_pop['Counties'].str.lower() == county_normalized]
    
    # Check if match was found
    if county_match.empty:
        print(f"No match found for county: {county_name} (normalized: {county_normalized})")
        return 0  # Return 0 instead of None
    
    # Return the population value
    return int(county_match['Total_Population'].iloc[0])
def get_car_prop(site, health_centers):
    county = get_county(site, health_centers)
    # Remove "County" from the end and convert to lowercase for matching
    county_normalized = county.lower().replace(" county", "").strip()
    
    # Find the car proportion for this county in county_pop
    county_match = county_pop[county_pop['Counties'].str.lower() == county_normalized]
    
    if county_match.empty:
        print(f"No match found for county: {county} (normalized: {county_normalized})")
        return 0.0  # Return a default value instead of None
    
    return county_match['Vehicle_Registration_Proportion'].iloc[0]


'''def get_county(site, health_centers): #changed from get_zip to get_county
    #return health_centers.loc[site]['county']
    county = get_county(site, health_centers)
    # Find the car proportion for this county in county_pop
    county_match = county_pop[county_pop['Counties'].str.lower() == county.lower()]
    if county_match.empty:
        print(f"No match found for county: {county}")
        return None
    return county_match['Vehicle_Registration_Proportion'].iloc[0]'''

def calc_dtilde_matrix(health_centers, t_car, t_pub, t_walk):
    N = health_centers.shape[0]
    dtilde_matrix = np.zeros((N,N))
    
    index_to_pos = {idx: pos for pos, idx in enumerate(health_centers.index)}
    
    for i, a in health_centers.iterrows():
        for j, b in health_centers.iterrows():
            if i != j:
                # for walking 
                pos_i = index_to_pos[i]
                pos_j = index_to_pos[j]
                
                if (t_walk.iloc[pos_i,pos_j] == 0 and t_walk.iloc[pos_j,pos_i] == 0):
                    walk = 0
                elif t_walk.iloc[pos_i,pos_j]==0:
                    walk=2*t_walk[pos_j,i]
                elif t_walk.iloc[pos_j,pos_i] == 0:
                    walk=2*t_walk.iloc[pos_i, pos_j]
                else: 
                    walk = t_walk.iloc[pos_i,pos_j] + t_walk.iloc[pos_j,pos_i] 
                # for public transport
                if t_pub.iloc[pos_i,pos_j] == 0 and t_pub.iloc[pos_j,pos_i] == 0:
                    pub = 0
                elif t_pub.iloc[pos_i,pos_j] == 0:
                    pub = 2*t_pub.iloc[pos_j,pos_i]
                elif t_pub.iloc[pos_j,pos_i] == 0:
                    pub = 2*t_pub.iloc[pos_i,pos_j]
                else:
                    pub = t_pub.iloc[pos_i,pos_j] + t_pub.iloc[pos_j,pos_i]
                # for car
                if t_car.iloc[pos_i,pos_j] == 0 and t_car.iloc[pos_j,pos_i] == 0:
                    car = 0
                elif t_car.iloc[pos_i,pos_j] == 0:
                    car = 2*t_car.iloc[pos_j,pos_i]
                elif t_car.iloc[pos_j,pos_i] == 0:
                    car=2*t_car.iloc[pos_i,pos_j]
                else:
                    car = t_car.iloc[pos_i,pos_j] + t_car.iloc[pos_j,pos_i]
                    
                driver_min = min(walk, pub, car)
                non_driver_min = min(walk, pub)
                car_prop = get_car_prop(i, health_centers)
                
                dtilde_matrix[pos_i,pos_j] = car_prop*driver_min + (1-car_prop)*non_driver_min
    return dtilde_matrix

def calc_d_matrix(dtilde_matrix, health_centers):
    N = health_centers.shape[0]
    d_matrix = np.zeros((N,N))
    index_to_pos = {idx: pos for pos, idx in enumerate(health_centers.index)}
    
    for i, a in health_centers.iterrows():
        for j, b in health_centers.iterrows():
            if i != j:
                pos_i = index_to_pos[i]
                pos_j = index_to_pos[j]
                county_i = get_county(i, health_centers)
                county_j = get_county(j, health_centers)
                
                #popi = get_pop(county_i)
                #popj = get_pop(county_j)
                popi = get_pop(county_i) # ERROR
                popj = get_pop(county_j)
                if popi == 0 and popj == 0:
                    d_matrix[pos_i,pos_j] = 0
                elif popi == 0:
                    d_matrix[pos_i,pos_j] = dtilde_matrix[pos_j,pos_i]
                elif popj == 0:
                    d_matrix[pos_i,pos_j] = dtilde_matrix[pos_i,pos_j]
                else:
                    d_matrix[pos_i,pos_j] = (1/(popi+popj))*(popi*dtilde_matrix[pos_i,pos_j] + popj*dtilde_matrix[pos_j,pos_i])

    return d_matrix

In [13]:
health_centers = pd.read_csv('facilities_w_county.csv',
                        index_col=0,
                        header=0)

### Distance Matrix for all 411 locations

In [ ]:
t_car = pd.read_csv("drive_matrix_all.csv",
                        index_col=0,
                        header=0)
t_pub = pd.read_csv("transit_matrix_all.csv",
                        index_col=0,
                        header=0)
# slc_twalk = calc_twalk_matrix(slc_sites, slc_neighbours)
t_walk = pd.read_csv("walking_matrix_all.csv",
                        index_col=0,
                        header=0)

In [54]:
all_loc_dtilde = calc_dtilde_matrix(health_centers, t_car, t_pub, t_walk)
all_loc_d_matrix = calc_d_matrix(all_loc_dtilde, health_centers)

### Loading matrix and saving to CSV for All Locations

In [55]:
np.save("all_loc_d_matrix_updated.npy", all_loc_d_matrix) 

In [ ]:
data = np.load("all_loc_d_matrix.npy")
df = pd.DataFrame(data)
print(df.head())
df = df.fillna(0)
print(df.head())
df.to_csv('dtilde_matrix_all.csv', index=False)

   0    1           2    3           4           5    6    7    8    9    ...  \
0  0.0  NaN         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
1  NaN  0.0         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
2  NaN  NaN    0.000000  NaN  903.671334         NaN  NaN  NaN  NaN  NaN  ...   
3  NaN  NaN         NaN  0.0         NaN         NaN  NaN  NaN  NaN  NaN  ...   
4  NaN  NaN  903.671334  NaN    0.000000  245.742626  NaN  NaN  NaN  NaN  ...   

   401         402        403  404  405  406  407  408  409         410  
0  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN  696.437903  
1  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
2  NaN  232.914218        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
3  NaN         NaN  88.047353  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
4  NaN  150.470140        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  

[5 rows x 411 columns]
   0    1           2    3           4       

### Distance Matrix for FQHCs only

In [ ]:
health_centers = pd.read_csv(r'facilities_w_county.csv', index_col = 0, header=0)
health_centers.index.tolist()
county_pop = pd.read_csv(r'county_vr_proportions.csv', index_col = 0, header = 0)

In [ ]:
# changing xlsx to csv 
#df = pd.read_excel("drive_minutes_matrix_fqhc.xlsx", index_col=0, header=0)
#df.to_csv("drive_minutes_matrix_fqhc.csv",index = True)
#data = pd.read_csv("drive_minutes_matrix_fqhc.csv", index_col=0, header=0)
#data = data.fillna(0)
data.to_csv("drive_matrix_fqhc.csv")
print(data.head())

       FQHC1  FQHC2      FQHC3  FQHC4      FQHC5      FQHC6  FQHC7  FQHC8  \
FQHC1    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC2    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC3    0.0    0.0   0.000000    0.0  72.633333  75.533333    0.0    0.0   
FQHC4    0.0    0.0   0.000000    0.0   0.000000   0.000000    0.0    0.0   
FQHC5    0.0    0.0  73.583333    0.0   0.000000  62.116667    0.0    0.0   

       FQHC9  FQHC10  ...  FQHC293  FQHC294    FQHC295    FQHC296  FQHC297  \
FQHC1    0.0     0.0  ...      0.0      0.0  52.766667   0.000000      0.0   
FQHC2    0.0     0.0  ...      0.0      0.0   0.000000   0.000000      0.0   
FQHC3    0.0     0.0  ...      0.0      0.0   0.000000  84.533333      0.0   
FQHC4    0.0     0.0  ...      0.0      0.0   0.000000   0.000000      0.0   
FQHC5    0.0     0.0  ...      0.0      0.0   0.000000  31.183333      0.0   

       FQHC298    FQHC299  FQHC300  FQHC301  Unnamed: 302  
FQHC1   

In [74]:
fqhc_t_car = pd.read_csv("drive_matrix_fqhc.csv",
                        index_col=0,
                        header=0)
fqhc_t_pub = pd.read_csv("transit_matrix_fqhc.csv",
                        index_col=0,
                        header=0)
fqhc_t_walk = pd.read_csv("walking_matrix_fqhc.csv",
                        index_col=0,
                        header=0)

In [75]:
fqhc_loc_dtilde = calc_dtilde_matrix(health_centers, t_car, t_pub, t_walk)
fqhc_loc_d_matrix = calc_d_matrix(fqhc_loc_dtilde, health_centers)

In [76]:
np.save("fqhc_d_matrix.npy", all_loc_d_matrix) 
data = np.load("fqhc_d_matrix.npy")
df = pd.DataFrame(data)
print(df.head())
df = df.fillna(0)
print(df.head())
df.to_csv('dtilde_matrix_fqhc.csv', index=False)

   0    1           2    3           4           5    6    7    8    9    ...  \
0  0.0  NaN         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
1  NaN  0.0         NaN  NaN         NaN         NaN  NaN  NaN  NaN  NaN  ...   
2  NaN  NaN    0.000000  NaN  903.671334         NaN  NaN  NaN  NaN  NaN  ...   
3  NaN  NaN         NaN  0.0         NaN         NaN  NaN  NaN  NaN  NaN  ...   
4  NaN  NaN  903.671334  NaN    0.000000  245.742626  NaN  NaN  NaN  NaN  ...   

   401         402        403  404  405  406  407  408  409         410  
0  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN  696.437903  
1  NaN         NaN        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
2  NaN  232.914218        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
3  NaN         NaN  88.047353  NaN  NaN  NaN  NaN  NaN  NaN         NaN  
4  NaN  150.470140        NaN  NaN  NaN  NaN  NaN  NaN  NaN         NaN  

[5 rows x 411 columns]
   0    1           2    3           4       

LAC

In [14]:
lac_sites = gpd.read_file('Los Angeles/fixed_files/lac_NoCatalina_polls.geojson')
lac_neighbours = np.load("Los Angeles/lac_neighbors.npy")
for i,a in lac_sites.iterrows():
    if a['address.zip'] == '90095':
        lac_sites.at[i, 'address.zip'] = '90024'
    elif a['address.zip'] == '90506':
        lac_sites.at[i, 'address.zip'] = '90249'
    elif a['address.zip'] == '91371':
        lac_sites.at[i, 'address.zip'] = '91367'

In [15]:
lac_tcar = np.load("Los Angeles/fixed_files/lac_tcar_matrix_completed.npy")
lac_tpub = np.load("Los Angeles/fixed_files/lac_tpub_matrix_completed.npy")
# lac_twalk = calc_twalk_matrix(lac_sites, lac_neighbours)
lac_twalk = np.load("walk_matrix_complete\lac_walk.npy")

In [16]:
lac_dtilde = calc_dtilde_matrix(lac_sites, lac_tcar, lac_tpub, lac_twalk, lac_neighbours)
lac_d_matrix = calc_d_matrix(lac_dtilde, lac_sites, lac_neighbours)
np.save("lac_d_matrix_updated.npy", lac_d_matrix) 

NYC

In [28]:
manbronx_sites = gpd.read_file('NYC/manbronx_polls.geojson')
manbronx_neighbours = np.load("NYC/manbronx_neighbors.npy")
for i,a in manbronx_sites.iterrows():
    if a['address.zip'] == '10103':
        manbronx_sites.at[i, 'address.zip'] = '10019'

manbronx_tcar = np.load("NYC/manbronx_tcar_matrix_completed.npy")
manbronx_tpub = np.load("NYC/manbronx_tpub_matrix_completed.npy")
# manbronx_twalk = calc_twalk_matrix(manbronx_sites, manbronx_neighbours)
manbronx_twalk = np.load("walk_matrix_complete\manbronx_walk.npy")

manbronx_dtilde = calc_dtilde_matrix(manbronx_sites, manbronx_tcar, manbronx_tpub, manbronx_twalk, manbronx_neighbours)
manbronx_d_matrix = calc_d_matrix(manbronx_dtilde, manbronx_sites, manbronx_neighbours)
np.save("manbronx_d_matrix_updated.npy", manbronx_d_matrix) 

In [32]:
queensbrook_sites = gpd.read_file('NYC/queensbrook_polls.geojson')
queensbrook_neighbours = np.load("NYC/queensbrook_neighbors.npy")

queensbrook_tcar = np.load("NYC/queensbrook_tcar_matrix_completed.npy")
queensbrook_tpub = np.load("NYC/queensbrook_tpub_matrix_completed.npy")
# queensbrook_twalk = calc_twalk_matrix(queensbrook_sites, queensbrook_neighbours)
queensbrook_twalk = np.load("walk_matrix_complete\queensbrook_walk.npy")

In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(queensbrook_sites['address.zip'])

0      11249
1      11233
2      11001
3      11001
4      11001
5      11001
6      11004
7      11005
8      11040
9      11040
10     11040
11     11040
12     11040
13     11101
14     11101
15     11104
16     11106
17     11101
18     11101
19     11101
20     11105
21     11102
22     11102
23     11102
24     11105
25     11103
26     11377
27     11104
28     11104
29     11377
30     11105
31     11106
32     11106
33     11101
34     11101
35     11109
36     11201
37     11201
38     11201
39     11201
40     11201
41     11201
42     11203
43     11203
44     11203
45     11203
46     11203
47     11203
48     11204
49     11204
50     11214
51     11204
52     11204
53     11204
54     11205
55     11205
56     11205
57     11205
58     11205
59     11206
60     11206
61     11206
62     11206
63     11206
64     11206
65     11206
66     11208
67     11207
68     11207
69     11207
70     11207
71     11212
72     11208
73     11208
74     11208
75     11208
76     11207

In [34]:
queensbrook_dtilde = calc_dtilde_matrix(queensbrook_sites, queensbrook_tcar, queensbrook_tpub, queensbrook_twalk, queensbrook_neighbours)
queensbrook_d_matrix = calc_d_matrix(queensbrook_dtilde, queensbrook_sites, queensbrook_neighbours)
np.save("queensbrook_d_matrix_updated.npy", queensbrook_d_matrix) 

In [35]:
stat_sites = gpd.read_file('NYC/stat_polls.geojson')
stat_neighbours = np.load("NYC/stat_neighbors.npy")

stat_tcar = np.load("NYC/stat_tcar_matrix_completed.npy")
stat_tpub = np.load("NYC/stat_tpub_matrix_completed.npy")
# stat_twalk = calc_twalk_matrix(stat_sites, stat_neighbours)
stat_twalk = np.load("walk_matrix_complete\stat_walk.npy")

stat_dtilde = calc_dtilde_matrix(stat_sites, stat_tcar, stat_tpub, stat_twalk, stat_neighbours)
stat_d_matrix = calc_d_matrix(stat_dtilde, stat_sites, stat_neighbours)
np.save("stat_d_matrix_updated.npy", stat_d_matrix) 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(over_18_pop_by_zip)

In [ ]:
np.save("slc_twalk_matrix_completed.npy", slc_twalk)
np.save("atl_twalk_matrix_completed.npy", atl_twalk)
np.save("lac_twalk_matrix_completed.npy", lac_twalk)
np.save("chc_twalk_matrix_completed.npy", chc_twalk)
np.save("jax_twalk_matrix_completed.npy", jax_twalk)
np.save("queensbrook_twalk_matrix_completed.npy", queensbrook_twalk)
np.save("stat_twalk_matrix_completed.npy", stat_twalk)
np.save("manbronx_twalk_matrix_completed.npy", manbronx_twalk)